# **3 机器学习：预测贸易流量 (40分)**

## **3.1 构建面板数据集 (10分)**

### 任务说明
按出口国-进口国-年份维度构建面板数据集：
1. 将HS6产品流量汇总为每年每对国家的双边出口总额
2. 按年份对面板数据排序
3. 按时间维度拆分（2016-2017年作为训练集，2018年作为测试集）
4. 以双边出口额v作为预测目标

### 步骤3.1.1：高效读取2016-2018年贸易数据并汇总

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# 使用相对路径，确保在不同协作者环境下都能正确读取
# 获取当前notebook所在目录的父目录
project_root = Path(os.getcwd()).parent if os.path.basename(os.getcwd()) == 'scripts' else Path(os.getcwd())
trade_data_dir = project_root / 'raw' / 'trade_data'

print(f"项目根目录: {project_root}")
print(f"贸易数据目录: {trade_data_dir}")
print(f"目录是否存在: {trade_data_dir.exists()}\n")

# 定义年份范围
years = [2016, 2017, 2018]

# 验证文件存在性
for year in years:
    filename = f'baci_hs12_y{year}_v202001.csv'
    filepath = trade_data_dir / filename
    if not filepath.exists():
        print(f"警告: 文件不存在 - {filepath}")
    else:
        print(f"✓ 找到文件: {filename}")

In [ ]:
%%time
# 高效读取和汇总策略：
# 1. 使用chunksize分块读取大文件，避免内存溢出
# 2. 在读取时立即进行分组汇总，减少内存占用
# 3. 只保留必需的列 (t, i, j, v)

print("=" * 80)
print("开始读取并汇总2016-2018年贸易数据...")
print("=" * 80)

# 存储每年汇总后的结果
aggregated_data = []

for year in years:
    filename = f'baci_hs12_y{year}_v202001.csv'
    filepath = trade_data_dir / filename
    
    print(f"\n正在处理 {year} 年数据: {filename}")
    
    # 分块读取并汇总，减少内存占用
    chunk_size = 500000  # 每次读取50万行
    chunks = []
    
    # 只读取需要的列
    usecols = ['t', 'i', 'j', 'k', 'v']
    
    for i, chunk in enumerate(pd.read_csv(filepath, usecols=usecols, 
                                          chunksize=chunk_size, low_memory=False)):
        # 在每个chunk内立即按 (t, i, j) 分组汇总
        chunk_agg = chunk.groupby(['t', 'i', 'j'], as_index=False)['v'].sum()
        chunks.append(chunk_agg)
        
        if (i + 1) % 5 == 0:
            print(f"  - 已处理 {(i + 1) * chunk_size:,} 行...")
    
    # 合并该年份所有chunk的汇总结果
    year_data = pd.concat(chunks, ignore_index=True)
    
    # 再次汇总（防止同一国家对在不同chunk中出现）
    year_data = year_data.groupby(['t', 'i', 'j'], as_index=False)['v'].sum()
    
    print(f"  ✓ {year}年数据汇总完成：共 {len(year_data):,} 个双边贸易流")
    aggregated_data.append(year_data)

# 合并所有年份数据
df_bilateral = pd.concat(aggregated_data, ignore_index=True)

print("\n" + "=" * 80)
print("数据汇总完成!")
print("=" * 80)
print(f"面板数据集总记录数: {len(df_bilateral):,}")
print(f"内存占用: {df_bilateral.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\n数据集基本信息:")
print(f"  - 年份范围: {df_bilateral['t'].min()} - {df_bilateral['t'].max()}")
print(f"  - 出口国数量: {df_bilateral['i'].nunique()}")
print(f"  - 进口国数量: {df_bilateral['j'].nunique()}")
print(f"  - 总贸易额: ${df_bilateral['v'].sum():,.0f} 千美元")
print(f"\n前5行数据:")
print(df_bilateral.head())

### 步骤3.1.2：数据质量检查和预处理

In [ ]:
# 数据质量检查
print("=" * 80)
print("数据质量检查")
print("=" * 80)

# 检查缺失值
print("\n1. 缺失值检查:")
print(df_bilateral.isnull().sum())

# 检查零值和负值
print("\n2. 贸易额统计:")
print(f"  - 零值记录数: {(df_bilateral['v'] == 0).sum():,}")
print(f"  - 负值记录数: {(df_bilateral['v'] < 0).sum():,}")
print(f"  - 最小值: ${df_bilateral['v'].min():,.2f}")
print(f"  - 最大值: ${df_bilateral['v'].max():,.2f}")
print(f"  - 平均值: ${df_bilateral['v'].mean():,.2f}")
print(f"  - 中位数: ${df_bilateral['v'].median():,.2f}")

# 移除异常值（负值，如果有的话）
if (df_bilateral['v'] < 0).any():
    print(f"\n警告: 发现负值，将被移除")
    df_bilateral = df_bilateral[df_bilateral['v'] >= 0].copy()

# 重命名列以更清晰
df_bilateral.columns = ['year', 'exporter', 'importer', 'export_value']

print("\n3. 每年的贸易流数量:")
for year in sorted(df_bilateral['year'].unique()):
    count = len(df_bilateral[df_bilateral['year'] == year])
    total_value = df_bilateral[df_bilateral['year'] == year]['export_value'].sum()
    print(f"  - {year}年: {count:,} 个贸易流, 总额 ${total_value:,.0f} 千美元")

print("\n数据预处理完成!")

### 步骤3.1.3：按年份排序并拆分训练集和测试集

In [ ]:
print("=" * 80)
print("按时间维度拆分数据集")
print("=" * 80)

# 按年份排序
df_bilateral = df_bilateral.sort_values(['year', 'exporter', 'importer']).reset_index(drop=True)

# 拆分训练集和测试集
# 训练集: 2016-2017年
# 测试集: 2018年
train_data = df_bilateral[df_bilateral['year'].isin([2016, 2017])].copy()
test_data = df_bilateral[df_bilateral['year'] == 2018].copy()

print("\n数据拆分结果:")
print("-" * 80)
print(f"训练集 (2016-2017年):")
print(f"  - 记录数: {len(train_data):,}")
print(f"  - 年份分布:")
for year in sorted(train_data['year'].unique()):
    count = len(train_data[train_data['year'] == year])
    pct = count / len(train_data) * 100
    print(f"    * {year}年: {count:,} ({pct:.1f}%)")
print(f"  - 总贸易额: ${train_data['export_value'].sum():,.0f} 千美元")
print(f"  - 平均贸易额: ${train_data['export_value'].mean():,.2f} 千美元")

print(f"\n测试集 (2018年):")
print(f"  - 记录数: {len(test_data):,}")
print(f"  - 总贸易额: ${test_data['export_value'].sum():,.0f} 千美元")
print(f"  - 平均贸易额: ${test_data['export_value'].mean():,.2f} 千美元")

print(f"\n数据集比例:")
print(f"  - 训练集占比: {len(train_data) / len(df_bilateral) * 100:.1f}%")
print(f"  - 测试集占比: {len(test_data) / len(df_bilateral) * 100:.1f}%")

# 查看目标变量分布
print(f"\n目标变量 (export_value) 分布:")
print(f"{'指标':<15} {'训练集':>20} {'测试集':>20}")
print("-" * 60)
print(f"{'最小值':<15} {train_data['export_value'].min():>20,.2f} {test_data['export_value'].min():>20,.2f}")
print(f"{'25分位数':<15} {train_data['export_value'].quantile(0.25):>20,.2f} {test_data['export_value'].quantile(0.25):>20,.2f}")
print(f"{'中位数':<15} {train_data['export_value'].median():>20,.2f} {test_data['export_value'].median():>20,.2f}")
print(f"{'75分位数':<15} {train_data['export_value'].quantile(0.75):>20,.2f} {test_data['export_value'].quantile(0.75):>20,.2f}")
print(f"{'最大值':<15} {train_data['export_value'].max():>20,.2f} {test_data['export_value'].max():>20,.2f}")
print(f"{'均值':<15} {train_data['export_value'].mean():>20,.2f} {test_data['export_value'].mean():>20,.2f}")
print(f"{'标准差':<15} {train_data['export_value'].std():>20,.2f} {test_data['export_value'].std():>20,.2f}")

print("\n" + "=" * 80)
print("数据拆分完成!")
print("=" * 80)

### 步骤3.1.4：数据集可视化和验证

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 设置绘图风格
sns.set_style("whitegrid")
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']  # 支持中文
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 创建图表
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('面板数据集概览', fontsize=16, fontweight='bold')

# 1. 每年的贸易流数量
ax1 = axes[0, 0]
year_counts = df_bilateral.groupby('year').size()
ax1.bar(year_counts.index, year_counts.values, color=['#3498db', '#2ecc71', '#e74c3c'])
ax1.set_xlabel('年份', fontsize=12)
ax1.set_ylabel('贸易流数量', fontsize=12)
ax1.set_title('每年双边贸易流数量', fontsize=13, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)
for i, v in enumerate(year_counts.values):
    ax1.text(year_counts.index[i], v, f'{v:,}', ha='center', va='bottom')

# 2. 训练集vs测试集
ax2 = axes[0, 1]
split_data = pd.DataFrame({
    '数据集': ['训练集\n(2016-2017)', '测试集\n(2018)'],
    '数量': [len(train_data), len(test_data)]
})
colors = ['#3498db', '#e74c3c']
bars = ax2.bar(split_data['数据集'], split_data['数量'], color=colors, alpha=0.7)
ax2.set_ylabel('记录数', fontsize=12)
ax2.set_title('训练集与测试集规模', fontsize=13, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}', ha='center', va='bottom')

# 3. 目标变量分布（对数尺度）
ax3 = axes[1, 0]
ax3.hist(np.log10(train_data['export_value'] + 1), bins=50, 
         alpha=0.7, label='训练集', color='#3498db', edgecolor='black')
ax3.hist(np.log10(test_data['export_value'] + 1), bins=50, 
         alpha=0.7, label='测试集', color='#e74c3c', edgecolor='black')
ax3.set_xlabel('log10(出口额 + 1)', fontsize=12)
ax3.set_ylabel('频数', fontsize=12)
ax3.set_title('目标变量分布（对数尺度）', fontsize=13, fontweight='bold')
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# 4. 每年总贸易额
ax4 = axes[1, 1]
year_values = df_bilateral.groupby('year')['export_value'].sum() / 1e6  # 转换为十亿美元
ax4.plot(year_values.index, year_values.values, marker='o', linewidth=2, 
         markersize=10, color='#2ecc71')
ax4.fill_between(year_values.index, year_values.values, alpha=0.3, color='#2ecc71')
ax4.set_xlabel('年份', fontsize=12)
ax4.set_ylabel('总出口额 (十亿美元)', fontsize=12)
ax4.set_title('每年总出口额趋势', fontsize=13, fontweight='bold')
ax4.grid(True, alpha=0.3)
for x, y in zip(year_values.index, year_values.values):
    ax4.text(x, y, f'${y:.1f}B', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\n可视化完成!")

### 步骤3.1.5：保存处理后的数据集

In [ ]:
# 创建输出目录
output_dir = project_root / 'results' / 'q3'
output_dir.mkdir(parents=True, exist_ok=True)

print("=" * 80)
print("保存处理后的数据集")
print("=" * 80)

# 保存完整的面板数据
panel_file = output_dir / 'q3_bilateral_panel_2016_2018.csv'
df_bilateral.to_csv(panel_file, index=False)
print(f"\n✓ 完整面板数据已保存: {panel_file}")
print(f"  文件大小: {panel_file.stat().st_size / 1024**2:.2f} MB")

# 保存训练集
train_file = output_dir / 'q3_train_data_2016_2017.csv'
train_data.to_csv(train_file, index=False)
print(f"\n✓ 训练集已保存: {train_file}")
print(f"  文件大小: {train_file.stat().st_size / 1024**2:.2f} MB")

# 保存测试集
test_file = output_dir / 'q3_test_data_2018.csv'
test_data.to_csv(test_file, index=False)
print(f"\n✓ 测试集已保存: {test_file}")
print(f"  文件大小: {test_file.stat().st_size / 1024**2:.2f} MB")

# 保存数据集摘要信息
summary_info = {
    '数据集': ['完整面板', '训练集', '测试集'],
    '年份范围': ['2016-2018', '2016-2017', '2018'],
    '记录数': [len(df_bilateral), len(train_data), len(test_data)],
    '总出口额(千美元)': [
        df_bilateral['export_value'].sum(),
        train_data['export_value'].sum(),
        test_data['export_value'].sum()
    ],
    '平均出口额(千美元)': [
        df_bilateral['export_value'].mean(),
        train_data['export_value'].mean(),
        test_data['export_value'].mean()
    ],
    '出口国数量': [
        df_bilateral['exporter'].nunique(),
        train_data['exporter'].nunique(),
        test_data['exporter'].nunique()
    ],
    '进口国数量': [
        df_bilateral['importer'].nunique(),
        train_data['importer'].nunique(),
        test_data['importer'].nunique()
    ]
}

df_summary = pd.DataFrame(summary_info)
summary_file = output_dir / 'q3_dataset_summary.csv'
df_summary.to_csv(summary_file, index=False)
print(f"\n✓ 数据集摘要已保存: {summary_file}")

print("\n" + "=" * 80)
print("数据集摘要:")
print("=" * 80)
print(df_summary.to_string(index=False))

print("\n" + "=" * 80)
print("第3.1题完成！")
print("=" * 80)
print("\n已成功完成:")
print("  ✓ 将HS6产品流量汇总为每年每对国家的双边出口总额")
print("  ✓ 按年份对面板数据排序")
print("  ✓ 按时间维度拆分（2016-2017年训练集，2018年测试集）")
print("  ✓ 以双边出口额作为预测目标")
print("  ✓ 数据已保存到 results/q3/ 目录")
print("\n准备进行下一步：构建机器学习模型")

### 步骤3.1.5：保存处理后的数据集

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 设置绘图风格
sns.set_style("whitegrid")
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']  # 支持中文
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 创建图表
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('面板数据集概览', fontsize=16, fontweight='bold')

# 1. 每年的贸易流数量
ax1 = axes[0, 0]
year_counts = df_bilateral.groupby('year').size()
ax1.bar(year_counts.index, year_counts.values, color=['#3498db', '#2ecc71', '#e74c3c'])
ax1.set_xlabel('年份', fontsize=12)
ax1.set_ylabel('贸易流数量', fontsize=12)
ax1.set_title('每年双边贸易流数量', fontsize=13, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)
for i, v in enumerate(year_counts.values):
    ax1.text(year_counts.index[i], v, f'{v:,}', ha='center', va='bottom')

# 2. 训练集vs测试集
ax2 = axes[0, 1]
split_data = pd.DataFrame({
    '数据集': ['训练集\n(2016-2017)', '测试集\n(2018)'],
    '数量': [len(train_data), len(test_data)]
})
colors = ['#3498db', '#e74c3c']
bars = ax2.bar(split_data['数据集'], split_data['数量'], color=colors, alpha=0.7)
ax2.set_ylabel('记录数', fontsize=12)
ax2.set_title('训练集与测试集规模', fontsize=13, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}', ha='center', va='bottom')

# 3. 目标变量分布（对数尺度）
ax3 = axes[1, 0]
ax3.hist(np.log10(train_data['export_value'] + 1), bins=50, 
         alpha=0.7, label='训练集', color='#3498db', edgecolor='black')
ax3.hist(np.log10(test_data['export_value'] + 1), bins=50, 
         alpha=0.7, label='测试集', color='#e74c3c', edgecolor='black')
ax3.set_xlabel('log10(出口额 + 1)', fontsize=12)
ax3.set_ylabel('频数', fontsize=12)
ax3.set_title('目标变量分布（对数尺度）', fontsize=13, fontweight='bold')
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# 4. 每年总贸易额
ax4 = axes[1, 1]
year_values = df_bilateral.groupby('year')['export_value'].sum() / 1e6  # 转换为十亿美元
ax4.plot(year_values.index, year_values.values, marker='o', linewidth=2, 
         markersize=10, color='#2ecc71')
ax4.fill_between(year_values.index, year_values.values, alpha=0.3, color='#2ecc71')
ax4.set_xlabel('年份', fontsize=12)
ax4.set_ylabel('总出口额 (十亿美元)', fontsize=12)
ax4.set_title('每年总出口额趋势', fontsize=13, fontweight='bold')
ax4.grid(True, alpha=0.3)
for x, y in zip(year_values.index, year_values.values):
    ax4.text(x, y, f'${y:.1f}B', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\n可视化完成！")

### 步骤3.1.4：数据集可视化和验证

In [ ]:
print("=" * 80)
print("按时间维度拆分数据集")
print("=" * 80)

# 按年份排序
df_bilateral = df_bilateral.sort_values(['year', 'exporter', 'importer']).reset_index(drop=True)

# 拆分训练集和测试集
# 训练集: 2016-2017年
# 测试集: 2018年
train_data = df_bilateral[df_bilateral['year'].isin([2016, 2017])].copy()
test_data = df_bilateral[df_bilateral['year'] == 2018].copy()

print("\n数据拆分结果:")
print("-" * 80)
print(f"训练集 (2016-2017年):")
print(f"  - 记录数: {len(train_data):,}")
print(f"  - 年份分布:")
for year in sorted(train_data['year'].unique()):
    count = len(train_data[train_data['year'] == year])
    pct = count / len(train_data) * 100
    print(f"    * {year}年: {count:,} ({pct:.1f}%)")
print(f"  - 总贸易额: ${train_data['export_value'].sum():,.0f} 千美元")
print(f"  - 平均贸易额: ${train_data['export_value'].mean():,.2f} 千美元")

print(f"\n测试集 (2018年):")
print(f"  - 记录数: {len(test_data):,}")
print(f"  - 总贸易额: ${test_data['export_value'].sum():,.0f} 千美元")
print(f"  - 平均贸易额: ${test_data['export_value'].mean():,.2f} 千美元")

print(f"\n数据集比例:")
print(f"  - 训练集占比: {len(train_data) / len(df_bilateral) * 100:.1f}%")
print(f"  - 测试集占比: {len(test_data) / len(df_bilateral) * 100:.1f}%")

# 查看目标变量分布
print(f"\n目标变量 (export_value) 分布:")
print(f"{'指标':<15} {'训练集':>20} {'测试集':>20}")
print("-" * 60)
print(f"{'最小值':<15} {train_data['export_value'].min():>20,.2f} {test_data['export_value'].min():>20,.2f}")
print(f"{'25分位数':<15} {train_data['export_value'].quantile(0.25):>20,.2f} {test_data['export_value'].quantile(0.25):>20,.2f}")
print(f"{'中位数':<15} {train_data['export_value'].median():>20,.2f} {test_data['export_value'].median():>20,.2f}")
print(f"{'75分位数':<15} {train_data['export_value'].quantile(0.75):>20,.2f} {test_data['export_value'].quantile(0.75):>20,.2f}")
print(f"{'最大值':<15} {train_data['export_value'].max():>20,.2f} {test_data['export_value'].max():>20,.2f}")
print(f"{'均值':<15} {train_data['export_value'].mean():>20,.2f} {test_data['export_value'].mean():>20,.2f}")
print(f"{'标准差':<15} {train_data['export_value'].std():>20,.2f} {test_data['export_value'].std():>20,.2f}")

print("\n" + "=" * 80)
print("数据拆分完成!")
print("=" * 80)

### 步骤3.1.3：按年份排序并拆分训练集和测试集

In [ ]:
# 数据质量检查
print("=" * 80)
print("数据质量检查")
print("=" * 80)

# 检查缺失值
print("\n1. 缺失值检查:")
print(df_bilateral.isnull().sum())

# 检查零值和负值
print("\n2. 贸易额统计:")
print(f"  - 零值记录数: {(df_bilateral['v'] == 0).sum():,}")
print(f"  - 负值记录数: {(df_bilateral['v'] < 0).sum():,}")
print(f"  - 最小值: ${df_bilateral['v'].min():,.2f}")
print(f"  - 最大值: ${df_bilateral['v'].max():,.2f}")
print(f"  - 平均值: ${df_bilateral['v'].mean():,.2f}")
print(f"  - 中位数: ${df_bilateral['v'].median():,.2f}")

# 移除异常值（负值，如果有的话）
if (df_bilateral['v'] < 0).any():
    print(f"\n警告: 发现负值，将被移除")
    df_bilateral = df_bilateral[df_bilateral['v'] >= 0].copy()

# 重命名列以更清晰
df_bilateral.columns = ['year', 'exporter', 'importer', 'export_value']

print("\n3. 每年的贸易流数量:")
for year in sorted(df_bilateral['year'].unique()):
    count = len(df_bilateral[df_bilateral['year'] == year])
    total_value = df_bilateral[df_bilateral['year'] == year]['export_value'].sum()
    print(f"  - {year}年: {count:,} 个贸易流, 总额 ${total_value:,.0f} 千美元")

print("\n数据预处理完成!")

### 步骤3.1.2：数据质量检查和预处理

In [ ]:
%%time
# 高效读取和汇总策略：
# 1. 使用chunksize分块读取大文件，避免内存溢出
# 2. 在读取时立即进行分组汇总，减少内存占用
# 3. 只保留必需的列 (t, i, j, v)

print("=" * 80)
print("开始读取并汇总2016-2018年贸易数据...")
print("=" * 80)

# 存储每年汇总后的结果
aggregated_data = []

for year in years:
    filename = f'baci_hs12_y{year}_v202001.csv'
    filepath = trade_data_dir / filename
    
    print(f"\n正在处理 {year} 年数据: {filename}")
    
    # 分块读取并汇总，减少内存占用
    chunk_size = 500000  # 每次读取50万行
    chunks = []
    
    # 只读取需要的列
    usecols = ['t', 'i', 'j', 'k', 'v']
    
    for i, chunk in enumerate(pd.read_csv(filepath, usecols=usecols, 
                                          chunksize=chunk_size, low_memory=False)):
        # 在每个chunk内立即按 (t, i, j) 分组汇总
        chunk_agg = chunk.groupby(['t', 'i', 'j'], as_index=False)['v'].sum()
        chunks.append(chunk_agg)
        
        if (i + 1) % 5 == 0:
            print(f"  - 已处理 {(i + 1) * chunk_size:,} 行...")
    
    # 合并该年份所有chunk的汇总结果
    year_data = pd.concat(chunks, ignore_index=True)
    
    # 再次汇总（防止同一国家对在不同chunk中出现）
    year_data = year_data.groupby(['t', 'i', 'j'], as_index=False)['v'].sum()
    
    print(f"  ✓ {year}年数据汇总完成：共 {len(year_data):,} 个双边贸易流")
    aggregated_data.append(year_data)

# 合并所有年份数据
df_bilateral = pd.concat(aggregated_data, ignore_index=True)

print("\n" + "=" * 80)
print("数据汇总完成!")
print("=" * 80)
print(f"面板数据集总记录数: {len(df_bilateral):,}")
print(f"内存占用: {df_bilateral.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\n数据集基本信息:")
print(f"  - 年份范围: {df_bilateral['t'].min()} - {df_bilateral['t'].max()}")
print(f"  - 出口国数量: {df_bilateral['i'].nunique()}")
print(f"  - 进口国数量: {df_bilateral['j'].nunique()}")
print(f"  - 总贸易额: ${df_bilateral['v'].sum():,.0f} 千美元")
print(f"\n前5行数据:")
print(df_bilateral.head())

### 步骤3.1.1：高效读取2016-2018年贸易数据并汇总

## **3.1 构建面板数据集 (10分)**

### 任务说明
按出口国-进口国-年份维度构建面板数据集：
1. 将HS6产品流量汇总为每年每对国家的双边出口总额
2. 按年份对面板数据排序
3. 按时间维度拆分（2016-2017年作为训练集，2018年作为测试集）
4. 以双边出口额v作为预测目标